<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/FastText_biGRU_crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [34]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [35]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-35-6ba94009a3f5>", line 1, in <module>
    get_ipython().magic('cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/')
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "</usr/local/lib/python3.6/dist-packages/decorator.py:decorator-gen-91>", line 2, in cd
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd(

OSError: ignored

In [0]:
ls

In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [0]:
train.head()

In [0]:
train.tail()

In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [0]:
import collections
collections.Counter(labels)

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
len(sequences_matrix),len(labels)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

In [0]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.tr.300.vec.gz', 'rb') as f_in:
#     with open('cc.tr.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.tr.300.vec', word_index)


In [0]:
embeddings.shape

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
  #model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(50, activation="relu"))
  model.add(Dense(20, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [0]:
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=2020)
cvscores = []
for train, test in kfold.split(sequences_matrix, labels):
  model=model_func()
  model.fit(sequences_matrix[train], labels[train],batch_size=256,epochs=10,verbose=2,class_weight=class_weights,
            validation_data=(sequences_matrix[test],labels[test]),callbacks=[reduce_lr])
  y_pred = model.predict(sequences_matrix[test], batch_size=128, verbose=1)
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[test], y_pred,average='macro')
  cvscores.append(f1)
  keras.backend.clear_session()